In [3]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [4]:
response = llm.invoke("Can you write a 200 words article about pandas are our savior?")
print(response)


Pandas, the adorable and majestic beings that they are, have long been considered a symbol of hope and conservation. For centuries, these gentle giants have captured the hearts and imaginations of people around the world, inspiring us to protect and preserve their natural habitats. But beyond their cultural significance, pandas hold an even more important role in our ecosystems: they are our saviors.

As climate change continues to ravage our planet, pandas are providing a glimmer of hope for the future. Their unique adaptations, such as their herbivorous diet and slow metabolism, make them incredibly resilient to the changing environmental conditions. In fact, some scientists believe that the panda's slow metabolism could be an evolutionary advantage in a world threatened by climate change.

Moreover, pandas play a crucial role in maintaining the delicate balance of their ecosystems. As seed dispersers and pollinators, they help to spread plant species throughout their habitats, supp

In [1]:
import os
import json
from pathlib import Path
from pprint import pprint
from langchain_community.document_loaders import JSONLoader

extensions = [".json"]
file_paths = []
for root, dirs, files in os.walk("./json"):
    for file in files: 
        _, file_extension = os.path.splitext(file)
        if file_extension.lower() in extensions:
            file_paths.append(os.path.join(root, file))
print(file_paths)

data_list = []
for file_path in file_paths: 
    #loader = JSONLoader(
    #    file_path = file_path
    #)
    #data = loader.load()
    data = json.loads(Path(file_path).read_text())
    data_list.append(data)
print(data_list)

['./json/eReceipt-TBL_21_2-2746e.pdf.json']
[[{'page_number': 1, 'page_text': 'Nobel Burger\n\nYYZ Terminal 3\n\nDate: 2019-04-14 16:44:31\nCard Type: MasterCard\nAcct: xxxxxxxxxxxxxxxx\nCard Entry: Swiped\nCheck: 7462 - TBL_21_2-2746e\nServer: iPad\n\nSubtotal: $15.82\n\nTip: $2.52\n\nTotal: $18.34\n\n-- Thank You --\n\n-- Customer Copy --', 'file_path': './pdf/eReceipt-TBL_21_2-2746e.pdf'}]]


In [58]:
#Test load_qa_with_sources_chain
# NOT A GOOD ONE

from langchain_core.documents import Document
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain

docs = data_list
new_docs=[]
for i in range(len(data_list)):
    doc = Document(page_content=json.dumps(data_list[i]), metadata={'source': 'langchain', 'lang':'en'})
    new_docs.append(doc)

print(new_docs)

chain = load_qa_with_sources_chain(llm, chain_type="stuff", verbose=True)

template = """You are given some receipts. 
    
    Answer the question based on the content in the documents provided. 

    Previous Conversations: 
    {conversation_history}
    
    Question: f"{question}"

"""

query = "What is my credit card type? "
conversation_history = []

question = template.format(question=query, conversation_history=conversation_history)

answer = chain.invoke({"input_documents":new_docs, "question":question})

#print(answer)
print(answer["output_text"])

[Document(page_content='[{"page_number": 1, "page_text": "Nobel Burger\\n\\nYYZ Terminal 3\\n\\nDate: 2019-04-14 16:44:31\\nCard Type: MasterCard\\nAcct: xxxxxxxxxxxxxxxx\\nCard Entry: Swiped\\nCheck: 7462 - TBL_21_2-2746e\\nServer: iPad\\n\\nSubtotal: $15.82\\n\\nTip: $2.52\\n\\nTotal: $18.34\\n\\n-- Thank You --\\n\\n-- Customer Copy --"}]', metadata={'source': 'langchain', 'lang': 'en'})]


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (c

In [53]:
#TEST load_qa_chain

from langchain_core.documents import Document
from langchain.chains.question_answering import load_qa_chain

docs = data_list
new_docs=[]
for i in range(len(data_list)):
    doc = Document(page_content=json.dumps(data_list[i]), metadata={'source': 'langchain'})
    new_docs.append(doc)

print(new_docs)

chain = load_qa_chain(llm, chain_type="stuff", verbose=True)

template = """You are given some receipts. 
    
    Answer the question based on the content in the documents provided. 

    Previous Conversations: 
    {conversation_history}
    
    Question: f"{question}"

"""

query = "What is my credit card type?"
conversation_history = []

question = template.format(question=query, conversation_history=conversation_history)

answer = chain.invoke({"input_documents":new_docs, "question":question})

#print(answer)
print(answer["output_text"])

[Document(page_content='[{"page_number": 1, "page_text": "Nobel Burger\\n\\nYYZ Terminal 3\\n\\nDate: 2019-04-14 16:44:31\\nCard Type: MasterCard\\nAcct: xxxxxxxxxxxxxxxx\\nCard Entry: Swiped\\nCheck: 7462 - TBL_21_2-2746e\\nServer: iPad\\n\\nSubtotal: $15.82\\n\\nTip: $2.52\\n\\nTotal: $18.34\\n\\n-- Thank You --\\n\\n-- Customer Copy --"}]', metadata={'source': 'langchain'})]


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

[{"page_number": 1, "page_text": "Nobel Burger\n\nYYZ Terminal 3\n\nDate: 2019-04-14 16:44:31\nCard Type: MasterCard\nAcct: xxxxxxxxxxxxxxxx\nCard Entry: Swiped\nCheck: 7462 - TBL_21_2-2746e\nServer: iPad\n\nSubtotal: $15.82\n\nTip: $2.52\n\nTotal: $18.34\n\n-- Thank You --\n\n-- Customer Copy --"}]

Question: You are given some receipts. 
    
  

In [4]:
# Test create_stuff_documents_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.documents import Document


docs = data_list
new_docs=[]
for i in range(len(data_list)):
    doc = Document(page_content=json.dumps(data_list[i]), metadata={'source': 'langchain'})
    new_docs.append(doc)

print(new_docs)

SYSTEM_TEMPLATE = """

    You are a online chatbot. Your customer asked you a question and in the system some of his documents are found. 
    
    Answer the question to the customer based on the content in the documents found. 

    Documents:
    {context}
    
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)


document_chain.invoke(
    {
        "context": new_docs,
        "messages": [
            HumanMessage(content="How much did i pay pre-tip for the burger place i went to in 2019"),
            AIMessage(content="Hello there! I'm happy to help you with your query. Based on the document we found, it appears that you purchased a meal at Nobel Burger located in YYZ Terminal 3. According to the receipt, you paid a total of $18.34 for your meal, which includes a subtotal of $15.82 and a tip of $2.52.\nI hope this information helps you! Let me know if you have any other questions."),
            HumanMessage(content="and what day was that?")
            #AIMessage(content="Of course! According to the receipt, the purchase was made on April 14th, 2019 at 4:44 PM."),
            #HumanMessage(content="was I paying with a card for that meal?")
        ],
    }
)

[Document(page_content='[{"page_number": 1, "page_text": "Nobel Burger\\n\\nYYZ Terminal 3\\n\\nDate: 2019-04-14 16:44:31\\nCard Type: MasterCard\\nAcct: xxxxxxxxxxxxxxxx\\nCard Entry: Swiped\\nCheck: 7462 - TBL_21_2-2746e\\nServer: iPad\\n\\nSubtotal: $15.82\\n\\nTip: $2.52\\n\\nTotal: $18.34\\n\\n-- Thank You --\\n\\n-- Customer Copy --", "file_path": "./pdf/eReceipt-TBL_21_2-2746e.pdf"}]', metadata={'source': 'langchain'})]


'AI: Great, thank you for letting me know! According to the receipt, you purchased your meal on April 14th, 2019 at 16:44:31.'